# Elasticsearch

In [2]:
import logging
import os
import sys

from dotenv import load_dotenv

load_dotenv("../../.env", override=True)

openai_token = os.environ["OPENAI_TOKEN"] 

os.environ["OPENAI_API_KEY"] = openai_token

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
#!pip install llama-index-embeddings-azure-openai
#!pip install llama-index-llms-azure-openai

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings


openai_chat_model = OpenAI()

openai_embedding_model = OpenAIEmbedding(
    model='text-embedding-3-small'
)

Settings.llm = openai_chat_model
Settings.embed_model = openai_embedding_model

## Create vector store

In [6]:
#!pip install llama-index-vector-stores-elasticsearch

In [5]:
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

vector_store = ElasticsearchStore(
    index_name="index", es_url="http://localhost:9200/"
)

INFO:elastic_transport.transport:GET http://localhost:9200/ [status:200 duration:0.016s]
GET http://localhost:9200/ [status:200 duration:0.016s]
GET http://localhost:9200/ [status:200 duration:0.016s]


## Create pipeline

In [6]:
import json

def load_descriptions_from_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            descriptions = [item['description'] for item in data if 'description' in item]
            return descriptions
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [7]:
from llama_index.core import Document
import json

text_list = load_descriptions_from_json('../../data/json/games.json')
documents = [Document(text=t) for t in text_list]

In [9]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        openai_embedding_model,
    ],
    vector_store=vector_store,
)

pipeline.run(documents=documents)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 0.789540 seconds
Retrying request to /embeddings in 0.789540 seconds
Retrying request to /embeddings in 0.789540 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 1.914673 seconds
Retrying request to /embeddings in 1.914673 seconds
Retrying request to /embeddings in 1.914673 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 T

KeyboardInterrupt: 

## Create search engines

In [11]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)

retriever_engine = index.as_retriever(sparse_top_k=5)

## Retrieve using vector search

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.034s]
POST http://localhost:9200/index/_search [status:200 duration:0.034s]
POST http://localhost:9200/index/_search [status:200 duration:0.034s]


In [14]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

def print_text_list(result):
    for text in result:
        print_markdown(text.text)
        print("--"*60)
        print()
        print()

def search_and_display(query):
    result = retriever_engine.retrieve(query)
    print_text_list(result)

In [15]:
search_and_display("A game that has 2 players and teaches us how to accept offers")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.021s]
POST http://localhost:9200/index/_search [status:200 duration:0.021s]
POST http://localhost:9200/index/_search [status:200 duration:0.021s]


Lets Not 














This improv game is in the following improv game categories: 
 Narration


 Description 

Excellent game to train narrative skills, but a bit controversial.



Play in pairs: start with a location (e.g. a beach).   One player give the suggestions (e.g. 
let`s find a talking crab).  The other player either accepts the offer, and both play accordingly, or refuses the offer.  When an offer is refused the first 
player needs to make a better offer.

 Notes 

This is a bit controversial in the sense that players may object to refusing
offers; after all, we teach players to accept offers.  
Explain that the goal
of this exercise is twofold:

 teach players to drop their own ideas (see  Chivalry ) and happily change course in their story
 teach players which offers are `fun` and which ones are not.  The player 
accepting or refusing the offers should accept any offer that `feels good` or 
that advances the story, and only reject offers that don`t sound like fun or don`t seem to offer any potential for an interesting story.



See  Ding and  Yes Lets - or Rather Not for similar exercises.

				
				
				

 
 Fortune Cookie 

					If you don`t like getting smooshed into other, stickier, smellier people, you`re in the wrong line of work -- Frank Oz
				











------------------------------------------------------------------------------------------------------------------------




Open Offer 














This improv game is in the following improv game categories: 
 Accepting
 Association
 Exercise


 Description 

One player starts a scene by making an offer that is not `complete`, in the
sense that it is vague enough not to define the who/what/where right away.
Another player steps in and completes the  Platform .


 Notes 

Although in performance we probably want to establish our  Platform as 
early as possible, this game teaches improv players how to quickly build platforms as a group or a team.  

				
				
				

 
 Fortune Cookie 

					90% of success is showing up --- Woody Allen 
				











------------------------------------------------------------------------------------------------------------------------




## Query using hybrid search
Denne kommer til å feile fordi vi ikke har Elasticsearch premium

In [ ]:
#response = retriever_hybrid_engine.retrieve("Hva er fartsgrensen i boliggater?")

## Query using HNSW EF Search

In [ ]:
# TODO Investigate how to use

## Query using ES client

In [16]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

In [17]:
# Define the query
query = {
    "query": {
        "match_all": {}
    },
    "size": 5  # Fetch 5 documents
}

# Replace 'your_index' with your actual index name
response = es.search(index="index", body=query)

# Print the results
for hit in response['hits']['hits']:
    print(hit['_source'])

INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.013s]
POST http://localhost:9200/index/_search [status:200 duration:0.013s]
POST http://localhost:9200/index/_search [status:200 duration:0.013s]
{'embedding': [-0.008651375770568848, -0.018825067207217216, 0.00288832257501781, -0.020129907876253128, -0.023337645456194878, 0.01646004058420658, -0.03517637029290199, -0.006476638372987509, -0.017085276544094086, -0.021122131496667862, 0.019205644726753235, 0.009874665178358555, -0.005249951034784317, -0.0025281317066401243, 0.00841351319104433, 0.0027014310471713543, 0.01580761931836605, -0.0014484088169410825, 0.007054302841424942, -0.03354531526565552, -0.005732470657676458, -0.00727177644148469, 0.003544141538441181, -0.012926092371344566, -0.01366686262190342, 0.0018655165331438184, 0.013945500366389751, -0.01369404699653387, 0.0007581846439279616, -0.01610664464533329, 0.02529490925371647, 0.0029172056820243597, 0.008556230925023556, -0.

### Keyword search

In [34]:
from IPython.display import display, Markdown

# Function to format and display each content
def display_content(contents):
    markdown_content = '\n\n---\n\n'.join(contents)  # Separating contents with horizontal lines
    display(Markdown(markdown_content))

def print_content(contents):
    for content in contents:
        print(content)


def print_es_response(elasticsearch_response):
    # Extract hits
    hits = elasticsearch_response.get('hits', {}).get('hits', [])
    
    # Extract and format content from each hit
    formatted_contents = []
    for hit in hits:
        content = hit.get('fields', {}).get('content', [''])[0]
        formatted_contents.append(content)

    display_content(formatted_contents)
    

In [19]:
def keyword_search(query):
    keyword_result = es.search(
        index="index",
        query={
            "match": {
                "content": {
                    "query": query,
                    "boost": 0.9
                }
            }
        },
        size=5,
        fields=["content"],
        _source=False
    )
    
    print()
    print()
    print_es_response(keyword_result)


In [20]:
keyword_search("2 players accept offers")

INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.050s]
POST http://localhost:9200/index/_search [status:200 duration:0.050s]
POST http://localhost:9200/index/_search [status:200 duration:0.050s]




Blind Line Offers 














This improv game is in the following improv game categories: 
 Exercise


 Description 

Line up the players, give`em each a number.  Call out 2 numbers.  Those 
players step on stage immediately, with a physical  Offer (do 
something, eat an apple, wipe the floor, whatever) as well as a vocal 
offer (say something).


The idea is that the players accept and explore each other`s offers
and figure out what the scene is about.

				
				
				

 
 Fortune Cookie 

					It is common sense to take a method and try it.  If it fails, admit it frankly and try another.  But above all, try something  --- Franklin D. Roosevelt
				











---

Lets Not 














This improv game is in the following improv game categories: 
 Narration


 Description 

Excellent game to train narrative skills, but a bit controversial.



Play in pairs: start with a location (e.g. a beach).   One player give the suggestions (e.g. 
let`s find a talking crab).  The other player either accepts the offer, and both play accordingly, or refuses the offer.  When an offer is refused the first 
player needs to make a better offer.

 Notes 

This is a bit controversial in the sense that players may object to refusing
offers; after all, we teach players to accept offers.  
Explain that the goal
of this exercise is twofold:

 teach players to drop their own ideas (see  Chivalry ) and happily change course in their story
 teach players which offers are `fun` and which ones are not.  The player 
accepting or refusing the offers should accept any offer that `feels good` or 
that advances the story, and only reject offers that don`t sound like fun or don`t seem to offer any potential for an interesting story.



See  Ding and  Yes Lets - or Rather Not for similar exercises.

				
				
				

 
 Fortune Cookie 

					If you don`t like getting smooshed into other, stickier, smellier people, you`re in the wrong line of work -- Frank Oz
				











---

Bidirectional Satellite TV 














This improv game is in the following improv game categories: 
 Exercise
 Look and Listen


 Description 

Fairly difficult combination of the  Satellite TV and  Satellite Radio improv games: players in pairs, establish a beat.
Every beat, player 1 offers a movement and player 2 offers a word of a story.  At the same time, players 
mirror each other, with a delay of 1 beat.

				
				
				

 
 Fortune Cookie 

					Nice People + Nice Choices = Boring Scenes --- Liz Allen/Jimmy Carrane
				











---

Doors 














This improv game is in the following improv game categories: 
 Environment
 Exercise
 Object Work


 Description 

Excellent introductory exercise for building environments and  Platform .  All players in 2 
rows.   2 facing
players play the exercise, the others watch.



Player 1 sets a (part of a) platform by miming a door.  As soon as player 2 has
an idea 
about where player 1 might be, player 2 steps into the scene and offers
a line of dialog, expressing where we are.  Player 2 answers player 1, 
acknowledging player 1`s offer.  This ends the scene.

 The audience should know after these 2 lines of dialog where the players
are, and who they are, or what their relationship is to one another.

 Notes 

`Doors` can be anything that separates one physical location from another.  
Players can mime real doors, but also gates, curtains, you name it.  Encourage 
players to try out as many types of `doors` as possible.  Use sound to 
enhance the platform, e.g. a door in a submarine makes a different kind of
noise than an elevator door.


See also  Three Line Environment .

				
				
				

 
 Fortune Cookie 

					If it stops being fun, stop doing it --- 
Bart Shattuck
				











---

Rhymes 














This improv game is in the following improv game categories: 
 Audience Participation
 Die
 Limitations
 Narration
 Performance
 Verbal wit


 Description 

Scene played in verses.  The idea is that the first player offers a line, and 
the second player rhymes to that.  Then the second player offers another line,
with which the first player needs to make a rhyme.


Players that hesitate or forget to rhyme `die` and are replaced by other players.  The idea is to keep the story going, so players that can`t find a good 
rhyme that would advance the story better die than screw up the story.

 See also  Verses .

				
				
				

 
 Fortune Cookie 

					If you don`t like getting smooshed into other, stickier, smellier people, you`re in the wrong line of work -- Frank Oz
				











### Vector search

In [21]:
def vector_search(query):
    query_embedding = azure_openai_embedding_model.get_text_embedding(query)

    response_vector = es.search(
        index="index",
        query={
            "knn": {
                "field": "embedding",  # Ensure this matches the name of your dense_vector field
                "query_vector": query_embedding,
                "num_candidates": 50,  # This is optional but can help with performance/accuracy trade-offs
                "boost": 0.1,
            }
        },
        size=5,
        fields=["content"],
        _source=False,
    )
    print()
    print()
    print_es_response(response_vector)

In [22]:
vector_search("A game that has 2 players and teaches us how to accept offers")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.015s]
POST http://localhost:9200/index/_search [status:200 duration:0.015s]
POST http://localhost:9200/index/_search [status:200 duration:0.015s]




Lets Not 














This improv game is in the following improv game categories: 
 Narration


 Description 

Excellent game to train narrative skills, but a bit controversial.



Play in pairs: start with a location (e.g. a beach).   One player give the suggestions (e.g. 
let`s find a talking crab).  The other player either accepts the offer, and both play accordingly, or refuses the offer.  When an offer is refused the first 
player needs to make a better offer.

 Notes 

This is a bit controversial in the sense that players may object to refusing
offers; after all, we teach players to accept offers.  
Explain that the goal
of this exercise is twofold:

 teach players to drop their own ideas (see  Chivalry ) and happily change course in their story
 teach players which offers are `fun` and which ones are not.  The player 
accepting or refusing the offers should accept any offer that `feels good` or 
that advances the story, and only reject offers that don`t sound like fun or don`t seem to offer any potential for an interesting story.



See  Ding and  Yes Lets - or Rather Not for similar exercises.

				
				
				

 
 Fortune Cookie 

					If you don`t like getting smooshed into other, stickier, smellier people, you`re in the wrong line of work -- Frank Oz
				











---

Open Offer 














This improv game is in the following improv game categories: 
 Accepting
 Association
 Exercise


 Description 

One player starts a scene by making an offer that is not `complete`, in the
sense that it is vague enough not to define the who/what/where right away.
Another player steps in and completes the  Platform .


 Notes 

Although in performance we probably want to establish our  Platform as 
early as possible, this game teaches improv players how to quickly build platforms as a group or a team.  

				
				
				

 
 Fortune Cookie 

					90% of success is showing up --- Woody Allen 
				











---

Blind Line Offers 














This improv game is in the following improv game categories: 
 Exercise


 Description 

Line up the players, give`em each a number.  Call out 2 numbers.  Those 
players step on stage immediately, with a physical  Offer (do 
something, eat an apple, wipe the floor, whatever) as well as a vocal 
offer (say something).


The idea is that the players accept and explore each other`s offers
and figure out what the scene is about.

				
				
				

 
 Fortune Cookie 

					It is common sense to take a method and try it.  If it fails, admit it frankly and try another.  But above all, try something  --- Franklin D. Roosevelt
				











---

Give and Take 














This improv game is in the following improv game categories: 
 Concentration
 Exercise


 Description 

This is a variation on  Walking by Numbers : all players find a position the room.  
 One person must always make sound and one person must walk. This can be the same person who is walking and making 
sound but it could be one person walking while a different person is making sound.

The focus can be given (i.e. a walker stops walking) or can be taken (i.e. someone else starts walking; the original walker must stop then).
Feel free to try with multiple walkers.

				
				
				

 
 Fortune Cookie 

					Your job is to make someone else`s shitty initation better, not introduce your own great idea --- Del Close
				











---

Goalie 














This improv game is in the following improv game categories: 
 Accepting
 Characters
 Exercise


 Description 

Great high-tempo exercise.  1 player up front.  He`s the goalie.  The other players all think of an opening
line for a scene, and a character.  When everyone has their opening line and character, we bombard the goalie
with these offers, one at a time.  Goalie needs to react right away to an offer, acknowledging the opening and
character, snap into an opposite character and reply 
to the opening.  Immediately after that the next player comes 
up with his or her offer.


This exercise is good for teaching players to react right away, and to snap into a character almost without thinking.

				
				
				

 
 Fortune Cookie 

					Our life is fritteren away by detail ... simplify! --- Henry Thoreau
				











### Hybrid søk

In [23]:
def hybrid_search(query, keyword):

    query_hybrid_embedding = azure_openai_embedding_model.get_text_embedding(query)
    query_hybrid_keyword = keyword
    
    response_hybrid = es.search(
        index="index",
        query={
            "match": {"content": {"query": query_hybrid_keyword, "boost": 0.9}},
        },
        knn={
            "field": "embedding",
            "query_vector": query_hybrid_embedding,
            "k": 5,
            "num_candidates": 50,
            "boost": 0.1,
        },
        fields=["content"],
        _source=False,
    )

    print()
    print()
    print_es_response(response_hybrid)

In [24]:
hybrid_search("A game that has 2 players and teaches us how to accept offers", "2 players accepting offers")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.032s]
POST http://localhost:9200/index/_search [status:200 duration:0.032s]
POST http://localhost:9200/index/_search [status:200 duration:0.032s]




Lets Not 














This improv game is in the following improv game categories: 
 Narration


 Description 

Excellent game to train narrative skills, but a bit controversial.



Play in pairs: start with a location (e.g. a beach).   One player give the suggestions (e.g. 
let`s find a talking crab).  The other player either accepts the offer, and both play accordingly, or refuses the offer.  When an offer is refused the first 
player needs to make a better offer.

 Notes 

This is a bit controversial in the sense that players may object to refusing
offers; after all, we teach players to accept offers.  
Explain that the goal
of this exercise is twofold:

 teach players to drop their own ideas (see  Chivalry ) and happily change course in their story
 teach players which offers are `fun` and which ones are not.  The player 
accepting or refusing the offers should accept any offer that `feels good` or 
that advances the story, and only reject offers that don`t sound like fun or don`t seem to offer any potential for an interesting story.



See  Ding and  Yes Lets - or Rather Not for similar exercises.

				
				
				

 
 Fortune Cookie 

					If you don`t like getting smooshed into other, stickier, smellier people, you`re in the wrong line of work -- Frank Oz
				











---

Goalie 














This improv game is in the following improv game categories: 
 Accepting
 Characters
 Exercise


 Description 

Great high-tempo exercise.  1 player up front.  He`s the goalie.  The other players all think of an opening
line for a scene, and a character.  When everyone has their opening line and character, we bombard the goalie
with these offers, one at a time.  Goalie needs to react right away to an offer, acknowledging the opening and
character, snap into an opposite character and reply 
to the opening.  Immediately after that the next player comes 
up with his or her offer.


This exercise is good for teaching players to react right away, and to snap into a character almost without thinking.

				
				
				

 
 Fortune Cookie 

					Our life is fritteren away by detail ... simplify! --- Henry Thoreau
				











---

Bidirectional Satellite TV 














This improv game is in the following improv game categories: 
 Exercise
 Look and Listen


 Description 

Fairly difficult combination of the  Satellite TV and  Satellite Radio improv games: players in pairs, establish a beat.
Every beat, player 1 offers a movement and player 2 offers a word of a story.  At the same time, players 
mirror each other, with a delay of 1 beat.

				
				
				

 
 Fortune Cookie 

					Nice People + Nice Choices = Boring Scenes --- Liz Allen/Jimmy Carrane
				











---

Blind Line Offers 














This improv game is in the following improv game categories: 
 Exercise


 Description 

Line up the players, give`em each a number.  Call out 2 numbers.  Those 
players step on stage immediately, with a physical  Offer (do 
something, eat an apple, wipe the floor, whatever) as well as a vocal 
offer (say something).


The idea is that the players accept and explore each other`s offers
and figure out what the scene is about.

				
				
				

 
 Fortune Cookie 

					It is common sense to take a method and try it.  If it fails, admit it frankly and try another.  But above all, try something  --- Franklin D. Roosevelt
				











---

Tug-O-War 














This improv game is in the following improv game categories: 
 Accepting
 Exercise
 Group


 Description 

Divide the class in 2 groups and have them mime a Tug-o-war fight.  See if

 the mimed rope does not stretch or shrink
 somebody wins (i.e. one group gladly looses (see  Chivalry )

 Variations 

Do 2 per 2.  Or try men 
against women.

				
				
				

 
 Fortune Cookie 

					Our life is fritteren away by detail ... simplify! --- Henry Thoreau
				











---

Doors 














This improv game is in the following improv game categories: 
 Environment
 Exercise
 Object Work


 Description 

Excellent introductory exercise for building environments and  Platform .  All players in 2 
rows.   2 facing
players play the exercise, the others watch.



Player 1 sets a (part of a) platform by miming a door.  As soon as player 2 has
an idea 
about where player 1 might be, player 2 steps into the scene and offers
a line of dialog, expressing where we are.  Player 2 answers player 1, 
acknowledging player 1`s offer.  This ends the scene.

 The audience should know after these 2 lines of dialog where the players
are, and who they are, or what their relationship is to one another.

 Notes 

`Doors` can be anything that separates one physical location from another.  
Players can mime real doors, but also gates, curtains, you name it.  Encourage 
players to try out as many types of `doors` as possible.  Use sound to 
enhance the platform, e.g. a door in a submarine makes a different kind of
noise than an elevator door.


See also  Three Line Environment .

				
				
				

 
 Fortune Cookie 

					If it stops being fun, stop doing it --- 
Bart Shattuck
				











---

Emotional Manipulation 














This improv game is in the following improv game categories: 
 Accepting
 Exercise
 Performance
 Trust


 Description 

This is a great  Accepting exercise, but it demands a certain level of  Trust . This can also be used as a technique during  Performance .

 The idea is to open a 2 player scene with a line of dialog that manipulates the other player's emotional state. This can be positive (Gee Robin, you look handsome today) or negative (Mary, have you been snacking again lately, I think you've gained even more weight).  Player 2 should accept this manipulation, and let it change the emotional state of her character.

 We mentioned  Trust : it should be clear to the players that we are manipulating the emotions of characters, not the players themselves.  Furthermore, the word 'manipulation' may have a negative connotation for some - this is merely a technique, not intended to make others feel bad/sad/manipulated.  Do not use this exercise unless the players have developed a great deal of trust; if this exercise makes a player uncomfortable, then stop the exercise. 

 Variations 

This can be done as a one-person exercise as well: have the player say or do something that will manipulate her own character's emotional state.

 Another variation is to blurt out dialog to which the emotional reaction cannot be predicted (Honey I'm pregnant) - but also in that case player 2 should react with a change in emotion.  E.g. happy: That's so wonderful baby, we've been trying for such a long time. Versus mad: You silly cow, don't tell me you forgot your birth control pills again. Versus sad: That makes me think of poor little Jimmy again, I miss him so much.

 See also  Pimping and  Shivving .
				
				
				

 
 Fortune Cookie 

					Fall, then figure out what to do on your way down ---  Del Close
				











---

Six Episodes 














This improv game is in the following improv game categories: 
 Accepting
 Exercise
 Group
 Narration


 Description 

Excellent game to teach group storytelling.  Give a big task, like `The launch of a new flavor of coke`, or 
`Building a new clubhouse for the girl scouts`.  The players get 1 minute to prepare 6 scenes, in which the given
task needs to be completed.  After 1 minute there is no more discussing, they just play the 6 scenes.



Time it and see if they can make their minds up.  Quarreling will just slow them down, so this 
will only work
if players agree; they should be immediately  Accepting any usable ides.



				
				
				

 
 Fortune Cookie 

					Train like you fight --- Skip Penk
				











---

Rhymes 














This improv game is in the following improv game categories: 
 Audience Participation
 Die
 Limitations
 Narration
 Performance
 Verbal wit


 Description 

Scene played in verses.  The idea is that the first player offers a line, and 
the second player rhymes to that.  Then the second player offers another line,
with which the first player needs to make a rhyme.


Players that hesitate or forget to rhyme `die` and are replaced by other players.  The idea is to keep the story going, so players that can`t find a good 
rhyme that would advance the story better die than screw up the story.

 See also  Verses .

				
				
				

 
 Fortune Cookie 

					If you don`t like getting smooshed into other, stickier, smellier people, you`re in the wrong line of work -- Frank Oz
				











---

Voices From Heaven 














This improv game is in the following improv game categories: 
 Characters
 Exercise
 Narration
 Performance


 Description 

Excellent game for building characters and group-storytelling.


4 players, ask the audience who they are, what their occupation would be.


These 4 characters have died together, and they are going to tell us how that came about.
Players line up.  The 
game is played in 3 rounds.  In every round, each player offers some information about how
they happened to die.  At the end of the thirds round, they should all be dead.

 Notes 

It`s fun to take risks - if in the first round each of the players just offers what they happened 
to be doing the day they died, and where.  It is quite a challenge to get these 4 stories together,
including the deaths of all characters, by the end of the third round. 
Also known as Death Con. .


See also  Goon River for a more open, more free  Long Form version 
of the same  Handle .


				
				
				

 
 Fortune Cookie 

					To err is human, to moo bovine --- NN
				











## Search options

### Keyword search

Search on specific matches on keywords:

`keyword_search("Keywords...")`

### Vector search

Search on input text intention match:

`vector_search("Input sentence...")`

### Hybrid search

Search on a combination of keyword match and sentence intent match:

`hybrid_search("Sentence intent...", "keywords...")`

In [30]:
vector_search("4 players performance")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.012s]
POST http://localhost:9200/index/_search [status:200 duration:0.012s]
POST http://localhost:9200/index/_search [status:200 duration:0.012s]


Emotional Quadrants 














This improv game is in the following improv game categories: 
 Limitations
 Performance


 Description 

Divide the scene in 4 quadrants, and allocate 4 different emotions to each quadrant.  Players improvize a scene, but need to take on the emotion of the quadrant they 

In [31]:
hybrid_search("A game for many players that are good for warm up", "warm up easy")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.021s]
POST http://localhost:9200/index/_search [status:200 duration:0.021s]
POST http://localhost:9200/index/_search [status:200 duration:0.021s]


Blind Line Up 














This improv game is in the following improv game categories: 
 Group
 Icebreakers
 Warm-up


 Description 

This one is like  Group Order , except that everyone keeps their eyes 
closed, and no talking is allowed while the group lines up.


 Notes 

 This is more a group exerc

In [35]:
vector_search("a game that help establish a platform")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index/_search [status:200 duration:0.021s]
POST http://localhost:9200/index/_search [status:200 duration:0.021s]
POST http://localhost:9200/index/_search [status:200 duration:0.021s]




Open Offer 














This improv game is in the following improv game categories: 
 Accepting
 Association
 Exercise


 Description 

One player starts a scene by making an offer that is not `complete`, in the
sense that it is vague enough not to define the who/what/where right away.
Another player steps in and completes the  Platform .


 Notes 

Although in performance we probably want to establish our  Platform as 
early as possible, this game teaches improv players how to quickly build platforms as a group or a team.  

				
				
				

 
 Fortune Cookie 

					90% of success is showing up --- Woody Allen 
				











---

Tossing 














This improv game is in the following improv game categories: 
 Group
 Trust


 Description 

Eight players form 2 lines, facing each other, and mesh their forearms together,
to form a platform to `launch` another player up in the air and catch him again.
Do make sure that your 
platform is sturdy (holding the other players by the wrists seems to be most solid).

 See also  Free Falling and  Heave Ho for similar games.

				
				
				

 
 Fortune Cookie 

					Success/failure is a side product of the approval/disapproval syndrome. Trying to succeed or giving in to failure drains us --- Viola Spolin
				











---

Three Line Environment 














This improv game is in the following improv game categories: 
 Environment
 Exercise
 Object Work


 Description 

Played with 2 players.  They get 3 lines (first player gets 2 lines, second one
 gets one line).  Using no more than 3 lines (A-B-A) they need to establish their
 Platform ; who they are, where they are, when and why.

 See also  Doors .

				
				
				

 
 Fortune Cookie 

					The only thing that makes life possible is permanent intolerable uncertainty - not knowing what comes next --- Ursula Guin
				











---

Disc(1) 














This improv game is in the following improv game categories: 
 Exercise
 Limitations


 Description 

Tell the players to imagine the stage is a disc, only supported right in the 
center of the stage.   We are going to play a scene, in which the disc needs 
to be balanced at all times.  Anytime a player moves, or a new player enters,
the others need to rebalance the disc, and every move needs to be justified.

 Notes 

Also known as Pivot.   
There is 
another version of this game - see  Disc(2) .

				
				
				

 
 Fortune Cookie 

					You`ve got to jump off cliffs all the time and build your wings on the way down. --- Ray Bradbury
				











---

Disc(2) 














This improv game is in the following improv game categories: 
 Exercise
 Limitations


 Description 

Imagine the stage is a large disc, supported in the middle.  The goal is to
keep the disc balanced.

  Game starts with a player entering the disc; he is the leader.  As soon
as the leader stops milling about on the disc, another player must jump in and
restore the balance.  The new player becomes the new leader, and the previous
leader becomes a follower.

 As soon as the balance is restored, the leader can start walking around; the other player stays where he is.  As soon as the leader stops again, the follower(s) move and restore the balance.  After that, the leader becomes one of the 
followers, and a new leader jumps in.

 The group of followers may split into groups of equal size, and restore the
balance that way.

 Keep in mind that followers don`t start moving until the leader stops.

 See also  Disc(1) .

				
				
				

 
 Fortune Cookie 

					Smooth seas do not make skillful sailors. --- African proverb
				









